In [17]:
# The blob account name = [accountname]
account_name = ""
# The name of the primary ADLS share
file_system_name=""
# The blob account key [iufquq34r423r2==] - used to generate a SAS key
account_key = ''



StatementMeta(small, 43, 2, Finished, Available)

In [18]:
# The input file name 
input_directory = f'abfss://{file_system_name}@{account_name}.dfs.core.windows.net/bbcsports/'
# The number of clusters - this can be automated or start with a guesstimate
number_of_clusters = 15
# The output directory where the output file will be written to
output_directory = f'abfss://{file_system_name}@{account_name}.dfs.core.windows.net/bbcsports/output/'
# The name of the output file
output_filename = 'bbcsports_BART_cosinepca2_euclidean.csv'

# The blob account url - https://[accountname].blob.core.windows.net
account_url = f"https://{account_name}.blob.core.windows.net"

# If this is set to True then the Coalesce notebook will need to be run to merge the partition files into a single file
LOW_MEMORY_MODE = False

# AML Experiment tracking

# Azure SubscriptionId
subscription_id=""
# AzureML Workspace Resource Group
resource_group=""
# AzureML Workspace Name
workspace_name=""

StatementMeta(small, 43, 3, Finished, Available)

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer, StopWordsRemover, PCA, RegexTokenizer
from pyspark.ml.clustering import LDA, KMeans, BisectingKMeans
from pyspark.ml import Pipeline
from pyspark import SparkContext, SparkConf
import sys
from pyspark.sql.functions import udf, col, size
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession
import ntpath
import os
from sklearn.manifold import TSNE
import numpy as np

from transformers import BartTokenizer, BartModel
model_name = 'facebook/bart-base'


StatementMeta(small, 43, 4, Finished, Available)

## Track the Experiment in Azure ML

In [ ]:
from azureml.core import Workspace, Experiment, Run
import mlflow

ws = Workspace(subscription_id=subscription_id, resource_group=resource_group, workspace_name=workspace_name)    
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
experiment_name = f"({mssparkutils.runtime.context['notebookname']}_{str(mssparkutils.env.getJobId())})"
mlflow.set_experiment(experiment_name)
mlflow.log_param("input_directory", input_directory)
mlflow.log_param("number_of_clusters", number_of_clusters)
mlflow.log_param("output_directory", output_directory)
mlflow.log_param("output_filename", output_filename)
mlflow.log_param("account_url", account_url)
mlflow.log_param("account_name", account_name)
mlflow.log_param("file_system_name", file_system_name)
mlflow.log_param("model_name", model_name)
mlflow.log_param("LOW_MEMORY_MODE", LOW_MEMORY_MODE)
params = {
    "sparkpool": mssparkutils.runtime.context['sparkpool'],
    "workspace": mssparkutils.runtime.context['workspace'],
    "notebookname": mssparkutils.runtime.context['notebookname'],
    "isForPipeline": mssparkutils.runtime.context['isForPipeline'],
    "pipelinejobid": mssparkutils.runtime.context['pipelinejobid']
}

mlflow.log_params(params)
mlflow.pyspark.ml.autolog()

In [20]:
# replace this with your folder data
df = spark.read.text(input_directory, wholetext=True)


StatementMeta(small, 43, 5, Finished, Available)

In [21]:
# remove rows with empty values  and reaname value column
import pyspark.sql.functions as F
df_filtered = df.filter(F.col("value")!="")
df_filtered = df_filtered.select(F.col("value").alias("text"))
df_filtered.printSchema()

StatementMeta(small, 43, 6, Finished, Available)

root
 |-- text: string (nullable = true)



In [22]:
# Extract the foldername 
from pyspark.sql.functions import regexp_extract, input_file_name
regex_str = "[\/]([^\/]+[\/][^\/]+)$"
df_updated = df_filtered.withColumn("filename", regexp_extract(input_file_name(),regex_str,1))

# Extract filename from directory
def extractfilename(str):
    nameStr = str.split('/') 
    return nameStr[1]

# Use the udf to update filename column
from pyspark.sql.functions import udf,col,lit
from pyspark.sql.types import StringType
udf_filename = udf(lambda x:extractfilename(x),StringType())
df_label = df_updated.withColumn("filename",udf_filename(col("filename")))
df_label.printSchema() 

StatementMeta(small, 43, 7, Finished, Available)

root
 |-- text: string (nullable = true)
 |-- filename: string (nullable = true)



In [24]:
import pyspark.sql.functions as F

df_label = df_label.withColumn("txt_array", F.split(F.regexp_replace("text", " ", ""), ','))

StatementMeta(small, 43, 9, Finished, Available)

In [58]:
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window
w = Window().orderBy(lit('A'))
df_label = df_label.withColumn("row_num", row_number().over(w))

StatementMeta(small, 48, 37, Finished, Available)

In [60]:
from pyspark.ml.feature import HashingTF, IDF
hashingTF = HashingTF(inputCol="txt_array", outputCol="tf",binary=True)
tf = hashingTF.transform(df_label)


StatementMeta(small, 48, 39, Finished, Available)

In [70]:
from pyspark.ml.feature import MinHashLSH
mh = MinHashLSH(inputCol="tf", outputCol="hashes", numHashTables=5)
model = mh.fit(tf)


StatementMeta(small, 48, 49, Finished, Available)

In [71]:
tf2 = tf

StatementMeta(small, 48, 50, Finished, Available)

In [74]:
results = model.approxSimilarityJoin(tf, tf2, 1, distCol="JaccardDistance")

results = results.withColumn('fname1',col('datasetA.filename')) \
                 .withColumn('fname2',col('datasetB.filename')) \
                 .withColumn('ID1',col('datasetA.row_num')) \
                 .withColumn('ID2',col('datasetB.row_num')) \
                 .withColumn('sim_score',1-col('JaccardDistance')) \
                 .filter(col('ID1')<col('ID2'))

StatementMeta(small, 48, 53, Finished, Available)

In [89]:
sample_text = spark.createDataFrame([(1,"rugby is a great sport")],["id","text"])
sample_text = sample_text.withColumn("txt_array", F.split(F.regexp_replace("text", " ", ""), ','))
sample_tf = hashingTF.transform(sample_text)
display(sample_tf)

StatementMeta(small, 48, 68, Finished, Available)

SynapseWidget(Synapse.DataFrame, 1fd46b7a-9d04-4540-b2bf-a3df16ca107a)

In [108]:
key = tf.select('tf').limit(1).collect()
key = key[0].__getattr__('tf')

StatementMeta(small, 48, 87, Finished, Available)

In [110]:
neighbors = model.approxNearestNeighbors(dataset=tf, key=key, numNearestNeighbors=10, distCol= 'JaccardDistance') 
display(neighbors)

StatementMeta(small, 48, 89, Finished, Available)

SynapseWidget(Synapse.DataFrame, 499651ed-1dd6-4a9b-aecd-d6dd7f96a209)

In [80]:
clusters = df_label.join(assignments,df_label.row_num==assignments.id)
display(clusters)

StatementMeta(small, 48, 59, Finished, Available)

SynapseWidget(Synapse.DataFrame, bc08d01e-a6ca-4394-b4b3-9d6b88dcdffd)